In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.metrics import * 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings(action='ignore')
import sys

## 데이터 정보

![](./data_explanation.png)

In [2]:
ROOT_PATH = "E:\\aivle\\2022_uplus_ai_ground\\data\\"

buy_data = pd.read_csv(ROOT_PATH + "buy_data.csv")
history_data = pd.read_csv(ROOT_PATH + "history_data.csv")
meta_data = pd.read_csv(ROOT_PATH + "meta_data.csv")
meta_data_plus = pd.read_csv(ROOT_PATH + "meta_data_plus.csv")
profile_data = pd.read_csv(ROOT_PATH + "profile_data.csv")
sample_submission = pd.read_csv(ROOT_PATH + "sample_submission.csv")
search_data = pd.read_csv(ROOT_PATH + "search_data.csv")
watch_e_data = pd.read_csv(ROOT_PATH + "watch_e_data.csv")

data_kor_explanation = ["구매 이력 데이터", "시청 시작 데이터", "콘텐츠 일반 메타 정보", "콘텐츠 확장 정보", "프로필 정보", "검색을 통한 시청 데이터", "시청 종료 데이터", "리더보드 제출 양식"]
data_str_list = ['buy_data', 'history_data', 'meta_data', 'meta_data_plus', 'profile_data', 'search_data', 'watch_e_data', 'sample_submission']
data_list = [buy_data, history_data, meta_data, meta_data_plus, profile_data, search_data, watch_e_data, sample_submission]

In [3]:
for i in range(len(data_list)):
    print('\033[96m' + '-'*65 + '\033[33m' + str(data_str_list[i]) + '\033[96m' + '-'*65)
    print('\033[37m' +data_kor_explanation[i])
    display(data_list[i].head())
    print(data_list[i].shape)
    print('\033[95m' + '-'*140)
    print(' ')


-----------------------------------------------------------------buy_data-----------------------------------------------------------------
구매 이력 데이터


,profile_id,ss_id,log_time,act_target_dtl,album_id,payment
0,255,20220426153942,20220426154128,MKID004,3767,500
1,1603,20220309163919,20220309164043,MKID004,1640,300
2,5303,20220411080324,20220411082725,MKID004,3821,500
3,5303,20220411080324,20220411084013,MKID004,3822,500
4,5303,20220411080324,20220411085607,MKID004,3823,500


(150, 6)
--------------------------------------------------------------------------------------------------------------------------------------------
 
-----------------------------------------------------------------history_data-----------------------------------------------------------------
시청 시작 데이터


,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N


(1005651, 8)
--------------------------------------------------------------------------------------------------------------------------------------------
 
-----------------------------------------------------------------meta_data-----------------------------------------------------------------
콘텐츠 일반 메타 정보


...

(6474, 5)
--------------------------------------------------------------------------------------------------------------------------------------------
 
-----------------------------------------------------------------watch_e_data-----------------------------------------------------------------
시청 종료 데이터


,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
0,3,20220301115653,20220301115805,MKID049,15,46,46,1
1,3,20220301115653,20220301115954,MKID049,16,104,105,1
2,3,20220301115653,20220301120114,MKID049,17,76,76,1
3,3,20220301115653,20220301120226,MKID049,18,67,68,1
4,3,20220301115653,20220301120400,MKID049,19,90,90,0


(892794, 8)
--------------------------------------------------------------------------------------------------------------------------------------------
 
-----------------------------------------------------------------sample_submission-----------------------------------------------------------------
리더보드 제출 양식


,profile_id,predicted_list
0,3,[0]
1,5,[0]
2,7,[0]
3,12,[0]
4,16,[0]


(8311, 2)
--------------------------------------------------------------------------------------------------------------------------------------------
 


In [4]:
data_list = [buy_data, history_data, meta_data, meta_data_plus, profile_data, search_data, watch_e_data, sample_submission]

for i in range (len(data_list)):
    print('\033[33m' + data_str_list[i] + '\033[39m' + " ")
    print(data_list[i].isna().sum())
    print('--'*20)

buy_data 
profile_id        0
ss_id             0
log_time          0
act_target_dtl    0
album_id          0
payment           0
dtype: int64
----------------------------------------
history_data 
profile_id              0
ss_id                   0
log_time                0
act_target_dtl          0
album_id                0
payment            932165
continuous_play         0
short_trailer           0
dtype: int64
----------------------------------------
meta_data 
album_id           0
title              0
sub_title          0
genre_large        0
genre_mid          0
genre_small    29183
country         8868
run_time           0
onair_date     37258
cast_1         14999
cast_2         20554
cast_3         26139
cast_4         30117
cast_5         36220
cast_6         39993
cast_7         41840
dtype: int64
----------------------------------------
meta_data_plus 
album_id         0
keyword_type     0
keyword_name     0
keyword_value    0
dtype: int64
----------------------------------

In [5]:
watch_test1 = watch_e_data.loc[:, ['album_id', 'profile_id']]
his_test1 = history_data.loc[:, ['album_id', 'profile_id']]
# display(watch_test1[watch_test1['profile_id'] == 3].head())
# display(his_test1[his_test1['profile_id'] == 3].head())
du_watch = watch_test1.drop_duplicates('profile_id')
du_his = his_test1.drop_duplicates('profile_id')
print('--'*30)
result1 = pd.concat([du_watch,du_his])
result2 = result1.drop_duplicates(['profile_id'], keep = False)
display(result2)

for i in range (653):
    print(result2.iloc[:, i])
    break
result3 = pd.concat([watch_e_data,result2])
result3 = result3.fillna(0)

# display(result3)
# result3.to_csv('watch_e_data_add_profile.csv')

------------------------------------------------------------


,album_id,profile_id
573,352,12
3472,2159,74
6085,16,136
11337,190,293
11370,625,300
...,...,...
999059,802,32627
999289,178,32647
999350,3326,32650
1005252,4027,32979


573         352
3472       2159
6085         16
11337       190
11370       625
           ... 
999059      802
999289      178
999350     3326
1005252    4027
1005480    1005
Name: album_id, Length: 653, dtype: int64


In [6]:
base_df = pd.DataFrame()
for i in range (653):
    a = result2.iloc[i, 1]
    if a in list(history_data['profile_id']):
        base_df = pd.concat([base_df,history_data[history_data['profile_id'] == a]])
display(base_df)
        

KeyboardInterrupt: 

In [ ]:
# display(watch_e_data.loc[watch_e_data['profile_id'] == 3])
his_pro = pd.DataFrame(history_data.groupby(['profile_id'])['album_id'].max())
display(his_pro)

pro_watch = pd.DataFrame(watch_e_data.groupby(['profile_id'])['watch_time', 'album_id'].max())
display(pro_watch)

,album_id
profile_id,
3,33
5,317
7,343
12,357
16,339
...,...
33022,1425
33023,3005
33026,6772


,watch_time,album_id
profile_id,,
3,120,32
5,1207,317
7,222,343
16,27,339
19,182,403
...,...,...
33019,719,124
33023,587,3005
33026,58,6772


In [ ]:
# for i in data_list:
#     if 'profile_id' in i.columns:
#         print('---'*50)
#         print(i['profile_id'].value_counts())
import numpy as np #numpy library
np.set_printoptions(threshold=np.inf, linewidth=np.inf) #inf = infinity 

sys.stdout = open('output.txt','w')
print(history_data['profile_id'].unique())
print('---'*50)
print(len(history_data['profile_id'].unique()))
print('+++++++++'*30)
print(watch_e_data['profile_id'].unique())
print('---'*50)
print(len(watch_e_data['profile_id'].unique()))

In [ ]:
groups = watch_e_data.groupby('profile_id')
groups.max('watch_time') #최대값

,ss_id,log_time,album_id,watch_time,total_time,continuous_play
profile_id,,,,,,
3,20220301115653,20220301123428,32,120,121,1
5,20220426111949,20220426115006,317,1207,1208,2
7,20220402114502,20220402131044,343,222,831,0
16,20220419185609,20220419190409,339,27,756,0
19,20220424140258,20220424141052,403,182,357,2
...,...,...,...,...,...,...
33019,20220424213917,20220424215210,124,719,731,1
33023,20220425103827,20220425113828,3005,587,607,2
33026,20220425172104,20220425172229,6772,58,58,1


In [ ]:
for i in range(len(data_str_list)):
    if 'profile_id' in data_list[i].columns:
        print("--"*40)
        print(data_str_list[i])
        print(data_kor_explanation[i])
        print(data_list[i].columns)

--------------------------------------------------------------------------------
buy_data
구매 이력 데이터
Index(['profile_id', 'ss_id', 'log_time', 'act_target_dtl', 'album_id',
       'payment'],
      dtype='object')
--------------------------------------------------------------------------------
history_data
시청 시작 데이터
Index(['profile_id', 'ss_id', 'log_time', 'act_target_dtl', 'album_id',
       'payment', 'continuous_play', 'short_trailer'],
      dtype='object')
--------------------------------------------------------------------------------
profile_data
프로필 정보
Index(['profile_id', 'sex', 'age', 'pr_interest_keyword_cd_1',
       'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
       'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2',
       'ch_interest_keyword_cd_3'],
      dtype='object')
--------------------------------------------------------------------------------
sample_submission
리더보드 제출 양식
Index(['profile_id', 'predicted_list'], dtype='object')
--------------------

In [ ]:
history_df = history_data.drop(['act_target_dtl'], axis=1)
watch_df = watch_e_data.drop(['act_target_dtl'], axis=1)

In [ ]:
data_list = [buy_data, history_data, meta_data, meta_data_plus, profile_data, sample_submission, search_data, watch_e_data]

buy_df2 = buy_data.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
history_df2 = history_df.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
search_df2 = search_data.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
watch_df2 = watch_df.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
# history_df2 = history_df.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
# history_df2 = history_df.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
# history_df2 = history_df.drop_duplicates().sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
history_df2['rating'] = 1
buy_df2['rating'] = 1
search_df2['rating'] = 1
watch_df2['rating'] = 1
history_df3 = history_df2.fillna(0)

display(history_df3, buy_df2, search_df2, watch_df2)

,profile_id,ss_id,log_time,album_id,payment,continuous_play,short_trailer,rating
0,3,20220301115653,20220301115719,15,0.0,Y,N,1
1,3,20220301115653,20220301115809,16,0.0,Y,N,1
2,3,20220301115653,20220301115958,17,0.0,Y,N,1
3,3,20220301115653,20220301120118,18,0.0,Y,N,1
4,3,20220301115653,20220301120229,19,0.0,Y,N,1
...,...,...,...,...,...,...,...,...
899268,33032,20220427155091,20220427155668,381,0.0,Y,N,1
899269,33032,20220427155091,20220427155680,381,0.0,Y,N,1
899270,33032,20220427155839,20220427155810,125,0.0,Y,N,1
899271,33032,20220427155706,20220427155838,125,0.0,Y,N,1


,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,rating
0,255,20220426153942,20220426154128,MKID004,3767,500,1
1,1603,20220309163919,20220309164043,MKID004,1640,300,1
2,5303,20220411080324,20220411082725,MKID004,3821,500,1
3,5303,20220411080324,20220411084013,MKID004,3822,500,1
4,5303,20220411080324,20220411085607,MKID004,3823,500,1
...,...,...,...,...,...,...,...
143,23882,20220421182514,20220421185231,MKID004,7441,300,1
144,23882,20220421182514,20220421185510,MKID004,8749,1200,1
145,31509,20220409115246,20220409115347,MKID004,19906,500,1
146,31837,20220427143436,20220427143618,MKID004,1640,300,1


,profile_id,ss_id,log_time,act_target_dtl,album_id,rating
0,59,20220303183951,20220303183914,MKID017,2141,1
1,59,20220304181433,20220304181525,MKID017,512,1
2,59,20220304181433,20220304182447,MKID017,2141,1
3,59,20220312071555,20220312071637,MKID017,2142,1
4,59,20220312071555,20220312081857,MKID017,2142,1
...,...,...,...,...,...,...
5820,32965,20220423161110,20220423164503,MKID017,23083,1
5821,32965,20220424065420,20220424065456,MKID017,15080,1
5822,32965,20220425073255,20220425073358,MKID017,15080,1
5823,32965,20220427062749,20220427063353,MKID017,16497,1


,profile_id,ss_id,log_time,album_id,watch_time,total_time,continuous_play,rating
0,3,20220301115653,20220301115805,15,46,46,1,1
1,3,20220301115653,20220301115954,16,104,105,1,1
2,3,20220301115653,20220301120114,17,76,76,1,1
3,3,20220301115653,20220301120226,18,67,68,1,1
4,3,20220301115653,20220301120400,19,90,90,0,1
...,...,...,...,...,...,...,...,...
800735,33032,20220427155091,20220427155653,381,463,464,1,1
800736,33032,20220427155091,20220427155694,381,462,463,1,1
800737,33032,20220427155839,20220427155826,125,10,520,0,1
800738,33032,20220427155706,20220427155836,125,6,521,0,1


In [ ]:
display(history_df2['log_time'].value_counts())
# display(watch_df2['profile_id'].value_counts())

20220304195233    6
20220305105701    5
20220319201805    5
20220314174023    5
20220304203828    5
                 ..
20220317172840    1
20220317175133    1
20220317175311    1
20220317175449    1
20220427155891    1
Name: log_time, Length: 797725, dtype: int64

In [ ]:
history_df.info()
watch_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005651 entries, 0 to 1005650
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   profile_id       1005651 non-null  int64  
 1   ss_id            1005651 non-null  int64  
 2   log_time         1005651 non-null  int64  
 3   album_id         1005651 non-null  int64  
 4   payment          73486 non-null    float64
 5   continuous_play  1005651 non-null  object 
 6   short_trailer    1005651 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 53.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892794 entries, 0 to 892793
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   profile_id       892794 non-null  int64
 1   ss_id            892794 non-null  int64
 2   log_time         892794 non-null  int64
 3   album_id         892794 non-null  int64
 4   watch_ti

In [ ]:
merge_test = pd.merge(meta_data,meta_data_plus, how = 'inner', on='album_id')

display(merge_test.head())

# merge_test = pd.merge(history_df3,watch_df2, how = 'inner', on='album_id')

# display(merge_test)
#################################################################################
# display(history_data['act_target_dtl'].unique())

# display(watch_e_data['act_target_dtl'].unique())

# merge_test = pd.concat([history_df3,watch_df2],axis=1)

# display(merge_test)

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7,keyword_type,keyword_name,keyword_value
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN,05010101,시리즈,0
1,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN,05010302,EBS,0
2,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN,05010401,3세,0
3,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN,05010402,4세,0
4,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN,05010407,1세,0


In [ ]:
display(meta_data.shape, meta_data.isna().sum())

(42602, 16)

album_id           0
title              0
sub_title          0
genre_large        0
genre_mid          0
genre_small    29183
country         8868
run_time           0
onair_date     37258
cast_1         14999
cast_2         20554
cast_3         26139
cast_4         30117
cast_5         36220
cast_6         39993
cast_7         41840
dtype: int64

In [ ]:
# merge_test2 = pd.merge(merge_test,search_data, how = 'inner', on='log_time')

# display(merge_test2)

display(merge_test.isna().sum(), merge_test['genre_small'].value_counts())
display( merge_test['country'].value_counts())
display(merge_test['onair_date'].value_counts())
display(merge_test['cast_1'].value_counts())
display(merge_test['cast_2'].value_counts())
display(merge_test['cast_3'].value_counts())
display(merge_test['cast_3'].value_counts())
display( merge_test['cast_4'].value_counts())
display( merge_test['cast_5'].value_counts())
display( merge_test['cast_6'].value_counts())
display( merge_test['cast_7'].value_counts())

album_id              0
title                 0
sub_title             0
genre_large           0
genre_mid             0
genre_small      551657
country          159748
run_time              0
onair_date       751710
cast_1           287452
cast_2           387465
cast_3           492979
cast_4           568757
cast_5           697339
cast_6           782857
cast_7           818688
keyword_type          0
keyword_name          0
keyword_value         0
dtype: int64

영어         138799
창의학습        56896
수학과학        17242
독서동화        16943
장난감놀이       14703
생활교육         8657
액션/모험        6953
예술교육         6737
한글           4028
기타           3595
EBS          3100
코믹/명랑        1353
순정/학원         725
한자            394
어린이/가족        346
체육안전          151
미술/전시          26
SF/판타지         26
클래식 콘서트        14
오페라            11
Name: genre_small, dtype: int64

한국       507714
영국        57672
미국        35211
중국        23768
이스라엘      15042
저지        11398
프랑스        5444
독일         3225
일본         3065
스위스        2828
핀란드        1953
이탈리아       1696
벨기에        1675
벨라루스       1087
크로아티아       291
네덜란드        194
캐나다         167
우크라이나       134
아르헨티나        36
오스트리아         8
Name: country, dtype: int64

20160322.0    373
20150501.0    252
20141001.0    202
20150508.0    192
20150529.0    189
             ... 
20110119.0      2
20110120.0      2
20110124.0      2
20130701.0      1
20080820.0      1
Name: onair_date, Length: 3052, dtype: int64

뽀로로     40617
핑크퐁     24313
키키      20115
폴리      16924
타요      15538
        ...  
소극장         9
큰화식조        9
알룩이         8
수도꼭지        8
큐피피         8
Name: cast_1, Length: 689, dtype: int64

...

엠버     15482
라니     12358
패디     10520
페이소     9662
루피      9461
       ...  
리버트       10
김민채       10
해님         8
트렁크        8
푸지         8
Name: cast_3, Length: 396, dtype: int64

루피      19542
헬리      17662
가니      11444
펑크       7130
에디       7027
        ...  
에드워드       12
피터         12
펭귄         11
토비         11
휴          10
Name: cast_4, Length: 304, dtype: int64

에디       11516
제스        7130
별이        6350
세찌        5897
크리스       3905
         ...  
살수차         14
로지          14
부두매니저       14
돌핀          11
험프          10
Name: cast_5, Length: 182, dtype: int64

뚝딱이     6350
바바      3123
구구      2340
우리      2082
샛별      2047
        ... 
빈치        15
스텔라       14
마마        10
험프         9
꼬마캐리       9
Name: cast_6, Length: 71, dtype: int64

비비고      3123
에이스      2047
꼬망       1481
루피       1345
누리        921
옥토        818
포포        673
멜빈        629
픽스        578
나잘난       500
듀크        439
하나        208
해피        187
마법사       147
네옹        143
쿱스        137
딩요        133
또봇 Y       95
보컬 강사      23
원호         23
교육         18
Name: cast_7, dtype: int64

In [ ]:
display(metge_test.shape, metge_test)

(40909171, 13)

,profile_id_x,ss_id,log_time_x,album_id_x,payment,continuous_play_x,short_trailer,profile_id_y,log_time_y,album_id_y,watch_time,total_time,continuous_play_y
0,3,20220301115653,20220301115719,15,NaN,Y,N,3,20220301115805,15,46,46,1
1,3,20220301115653,20220301115719,15,NaN,Y,N,3,20220301115954,16,104,105,1
2,3,20220301115653,20220301115719,15,NaN,Y,N,3,20220301120114,17,76,76,1
3,3,20220301115653,20220301115719,15,NaN,Y,N,3,20220301120226,18,67,68,1
4,3,20220301115653,20220301115719,15,NaN,Y,N,3,20220301120400,19,90,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40909166,33032,20220427155091,20220427155680,381,NaN,Y,N,33032,20220427155653,381,463,464,1
40909167,33032,20220427155091,20220427155680,381,NaN,Y,N,33032,20220427155694,381,462,463,1
40909168,33032,20220427155839,20220427155810,125,NaN,Y,N,33032,20220427155826,125,10,520,0
40909169,33032,20220427155839,20220427155810,125,NaN,Y,N,33032,20220427155897,381,462,464,1


In [ ]:
display(history_data, watch_e_data)
for i in range (50):
    print(history_data['album_id'][i], ',', end='')
    print(watch_e_data['album_id'][i])

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N
1005648,33032,20220427155839,20220427155810,MKID003,125,NaN,Y,N
1005649,33032,20220427155706,20220427155838,MKID003,125,NaN,Y,N


,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
0,3,20220301115653,20220301115805,MKID049,15,46,46,1
1,3,20220301115653,20220301115954,MKID049,16,104,105,1
2,3,20220301115653,20220301120114,MKID049,17,76,76,1
3,3,20220301115653,20220301120226,MKID049,18,67,68,1
4,3,20220301115653,20220301120400,MKID049,19,90,90,0
...,...,...,...,...,...,...,...,...
892789,33032,20220427155091,20220427155653,MKID049,381,463,464,1
892790,33032,20220427155091,20220427155694,MKID049,381,462,463,1
892791,33032,20220427155839,20220427155826,MKID049,125,10,520,0
892792,33032,20220427155706,20220427155836,MKID049,125,6,521,0


15 ,15
16 ,16
17 ,17
18 ,18
19 ,19
20 ,20
21 ,21
22 ,22
23 ,23
24 ,24
25 ,25
26 ,26
27 ,27
28 ,28
29 ,29
30 ,30
31 ,31
32 ,32
20 ,20
21 ,21
33 ,65
65 ,66
66 ,67
67 ,68
68 ,65
65 ,67
67 ,69
69 ,70
70 ,71
71 ,72
72 ,74
73 ,75
74 ,74
75 ,75
74 ,76
75 ,77
76 ,78
77 ,79
78 ,80
79 ,81
80 ,82
81 ,83
82 ,84
83 ,85
84 ,86
85 ,87
86 ,88
87 ,89
88 ,90
89 ,91
